In [18]:
#all imports used
import requests
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
from sklearn.model_selection import train_test_split

In [34]:
def get_crypto_data(crypto_name):
    
    #sources API coingecko
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_name}/market_chart?vs_currency=usd&days=max&interval=daily"
    response = requests.get(url)
    data = response.json()['prices']
    
    #creates DataFrame with columns date and price
    df = pd.DataFrame(data, columns=['date', 'price'])
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    df.set_index('date', inplace=True)
    df.index = df.index.date

    df = df.groupby(df.index).max()
    return df

In [35]:
def price_prediction(df):
    
    # Prepare data
    price_data = df['price'].values

    #How many previous prices the model will use to predict the target
    past_days = 7
    
    X = []
    y = []
    for i in range(len(price_data) - past_days):
        X.append(price_data[i:i+past_days])
        y.append(price_data[i+past_days])

    X = np.array(X)
    y = np.array(y)
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    
    #Leave 20% to test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # Set up LSTM model
    model = models.Sequential()
    model.add(layers.LSTM(64, activation='relu', input_shape = (past_days, 1)))
    model.add(layers.Dense(32, activation = "relu"))
    model.add(layers.Dense(1, activation='linear'))

    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics = ["mae"])

    history = model.fit(X_train, y_train,
                            validation_split = 0.3,
                            shuffle = False,
                            batch_size = 32,
                            epochs = 15,
                            verbose = 0)
    
    

    # Get the last few prices, dependent on past_days
    latest_prices = df['price'].values[-past_days:]
    X = np.append(X, latest_prices)[-past_days:]
    
    # Reshape X to match the input shape of the LSTM model
    X = X.reshape((1, past_days, 1))

    
    # Use the trained model to predict the next price
    next_price = model.predict(X, verbose = 0)[0][0]
    
    return next_price

In [36]:
#returns DataFrame for prediction, passes in cryptocurrency
df = get_crypto_data("bitcoin")

#returns predicted price for next day, passes in DataFrame
prediction = price_prediction(df)